## Emotion Recognition Project

2019 - OT.

Given the Karolinska faces collection, train a classifier to predict emotion.
Details about the data (KDEF):
70 subjects (35 male, 35 female), all white, between 20 and 30 years of age, photographed with 7 emotion expressions:
neutral, happy, angry, afraid, disgusted, sad, surprised

Files stored in KDEF folder. Image naming code:
Codes:
	Example: AF01ANFL.JPG
		Letter 1: Session 
					A = series one
					B = series two
		Letter 2: Gender 
					F = female
					M = male
		Letter 3 & 4: Identity number
					01 - 35
		Letter 5 & 6: Expression
					AF = afraid
					AN = angry
					DI = disgusted
					HA = happy
					NE = neutral
					SA = sad
					SU = surprised
		Letter 7 & 8: Angle
					FL = full left profile
					HL = half left profile
					S = straight
					HR = half right profile
					FR = full right profile


In [1]:

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

Import the libraries needed for a CNN model 

In [2]:
from keras.preprocessing.image import ImageDataGenerator 
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 
from keras.preprocessing.image import load_img

Using TensorFlow backend.


Take the data from KDEF and copy samples in a data tree structured into a train set (with 290 images per emotion, focusing right now only on fearful and happy), and a validation set with 60 images per emotion. 

In [3]:
import os
import fnmatch
import shutil

#Initialize the number of images in class:
k = 0
afraid = r"[A-Z][A-Z][0-9][0-9]AF*.JPG"
happy = r"[A-Z][A-Z][0-9][0-9]HA*.JPG"
basedir = 'data/train/happy/'
# List all files in the female directories with angry expressions using scandir()
for i in range(1,36):
    if i < 10:
        n = '0' + str(i)
    else:
        n = str(i)   
    basepath = 'KDEF/KDEF_and_AKDEF/KDEF/AF' + n + '/'
    with os.scandir(basepath) as entries:
        for entry in entries:
            if entry.is_file():
                if fnmatch.fnmatch(entry.name, happy):
                    #print(entry.name)
                    old_name = os.path.join(basepath, entry.name)
                    #print(old_name)
                     # Initial new name
                    new_name = os.path.join(basedir,entry.name)
                    #print(new_name)
                    shutil.copy(old_name, new_name)
                    
                    k += 1
                    
# List all files in the male directories with angry expressions using scandir()                    
for i in range(1,36):
    if i < 10:
        n = '0' + str(i)
    else:
        n = str(i)   
    basepath = 'KDEF/KDEF_and_AKDEF/KDEF/AM' + n + '/'
    with os.scandir(basepath) as entries:
        for entry in entries:
            if entry.is_file():
                if fnmatch.fnmatch(entry.name, happy):
                    #print(entry.name)
                    old_name = os.path.join(basepath, entry.name)
                    #print(old_name)
                     # Initial new name
                    new_name = os.path.join(basedir,entry.name)
                    #print(new_name)
                    shutil.copy(old_name, new_name)
                    k += 1
print(k)

350


Image size and input shape:


In [3]:
img_width = 562
img_height = 762

In [4]:
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

Assign the train data directory, validation directory, and epochs and batch size for the CNN.
There are 290 images per emotion in the train set, 580 in total, and 60 images per emotion in the validation set, for a total of 120 images. 


In [5]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples =580 
nb_validation_samples = 120
epochs = 10
batch_size = 16


In [6]:
model = Sequential() 
model.add(Conv2D(32, (2, 2), input_shape = input_shape)) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size =(2, 2))) 
  
model.add(Conv2D(32, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size =(2, 2))) 
  
model.add(Conv2D(64, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size =(2, 2))) 
  
model.add(Flatten()) 
model.add(Dense(64)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(1)) 
model.add(Activation('sigmoid')) 
  
model.compile(loss ='binary_crossentropy', 
                     optimizer ='rmsprop', 
                   metrics =['accuracy']) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
train_datagen = ImageDataGenerator( 
    rescale=1. / 255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True) 
  
test_datagen = ImageDataGenerator(rescale=1. / 255) 

In [8]:
train_generator = train_datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='binary') 
  
validation_generator = test_datagen.flow_from_directory( 
    validation_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='binary') 

Found 640 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [9]:
model.fit_generator( 
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size, 
    epochs=epochs, 
    validation_data=validation_generator, 
    validation_steps=nb_validation_samples // batch_size) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
36/36 [==============================] - 89s 2s/step - loss: 1.0198 - acc: 0.5469 - val_loss: 0.6881 - val_acc: 0.5714
Epoch 2/10
36/36 [==============================] - 83s 2s/step - loss: 0.6993 - acc: 0.5295 - val_loss: 0.6935 - val_acc: 0.4712
Epoch 3/10
36/36 [==============================] - 83s 2s/step - loss: 0.7239 - acc: 0.5486 - val_loss: 0.6822 - val_acc: 0.7115
Epoch 4/10
36/36 [==============================] - 83s 2s/step - loss: 0.7106 - acc: 0.5191 - val_loss: 0.7370 - val_acc: 0.4615
Epoch 5/10
36/36 [==============================] - 83s 2s/step - loss: 0.7074 - acc: 0.5608 - val_loss: 0.6141 - val_acc: 0.6442
Epoch 6/10
36/36 [==============================] - 85s 2s/step - loss: 0.6536 - acc: 0.6233 - val_loss: 0.6267 - val_acc: 0.6731
Epoch 7/10
36/36 [==============================] - 97s 3s/step - loss: 0.6478 - acc: 0.6424 - val_loss: 0.4813 - val_acc: 0.8077
Epoch 8/10
36/36 [========================